In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import argparse
import sys,os
import tensorflow as tf
from tqdm import tqdm
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
import glob
import re
import numpy as np

c:\users\nived\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv('D:\\kaggle\\ndsc\\Submission2.csv')
df.head()
df.shape


(1174802, 2)

In [16]:
df_p = pd.read_csv('D:\\kaggle\\ndsc\\Submission2.csv')
#df_p.head()
df_p.shape
print(df_p.loc[df['id']=='1529179598_Brand','tagging'].values)

['45 29']


In [15]:
fIN = pd.read_csv('D:\\kaggle\\ndsc\\Image-classification-transfer-learning\\submission_image.csv')
fIN.shape
print(fIN.loc[df['id']=='1529179598_Brand','tagging'].values)

['45 39']


In [3]:
def change_val(id,x,df):
    val = df.loc[df['id']==id,'tagging']
    if val.empty:
        add_row(df,[id,x])
        print(df.loc[df['id']==id,'tagging'])
    else:
        v1 = val.values[0].split(' ')[0]
        df.loc[df['id']==id,'tagging'] = str(v1)+ " " + str(x)

In [135]:
df2 = pd.DataFrame([[1, 2], [3, 4]], columns=list('AB'))
df2= df2.append([{'B':10.0}], ignore_index=True)
df2.head()
#dt
#dt.append([{'B':10.0}], ignore_index=True)
#dt

c:\users\nived\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:5186: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  other = other.loc[:, self.columns]
c:\users\nived\appdata\local\programs\python\python36\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,A,B
0,1.0,2.0
1,3.0,4.0
2,NaN,10.0


In [4]:
def add_row(df, row):
    df.loc[-1] = row
    df.index = df.index + 1  
    return df


In [62]:
#check = df['id'].str.contains("637234604_Benefits")
#x = df[check]
#x['tagging'] = '2 3'
#x = df.loc[df['id']=='637234604_Benefits','tagging']
#df.loc[df['id']=='637234604_Benefits','tagging'] = '2 3'
#df.loc[df['id']=='637234604_Benefits','tagging']

1    2 3
Name: tagging, dtype: object

In [5]:
t=tqdm(pd.read_csv('mobile_data_info_val_competition.csv').values)
test_id=[]
test_path=[]
tot=0
for tt in t:
    test_id.append(tt[0])
    test_path.append(tt[2])
    tot+=1
    

100%|███████████████████████████████████████████████████████████████████████| 40417/40417 [00:00<00:00, 1013161.59it/s]


In [24]:
print(test_path[1])
print(test_id[1])
print(str(test_id[2])+";")
print(tot)
lkh = 4
for j in range(lkh):
    print(j)

mobile_image/e2b902b7cd35cd50f061d8e2a3ba7178.jpg
1520516704
1520726573;
40417
0
1
2
3


In [6]:
def load_image(filename):
    #Read in the image_data to be classified."""
    return tf.gfile.FastGFile(filename, 'rb').read()

def load_labels(filename):
    #Read in labels, one label per line."""
    return [line.rstrip() for line in tf.gfile.GFile(filename)]

def load_graph(filename):
    #Unpersists graph from file as default graph."""
    with tf.gfile.FastGFile(filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

In [7]:
def read_text(file):
    im = Image.open(file) # the second one 
    im = im.filter(ImageFilter.MedianFilter())
    enhancer = ImageEnhance.Contrast(im)
    im = enhancer.enhance(2)
    im = im.convert('1')
    im.save('temp2.jpg')
    x = -1
    text = pytesseract.image_to_string(Image.open('temp2.jpg')).lower()
    if "google" in text or "pixel" in text:
        x = 0
    elif "htc" in text:
        x = 1
    elif "apple" in text or "iphone" in text or "ipod" in text or "ipad" in text:
        x = 2
    elif "wiko" in text:
        x = 3
    elif "polytron" in text:
        x = 4
    elif "gionee" in text or "x1" in text or "m2017" in text or "m7" in text or "s11" in text or "w909" in text or "s10" in text:
        x = 5
    elif "leagoo" in text:
        x = 6
    elif "brandcode" in text:
        x = 7
    elif "luna" in text:
        x =8
    elif "acer" in text or "aspire" in text:
        x = 9
    elif "sharp" in text:
        x = 10
    elif "blackview" in text or bool(re.match(r"bv[0-9]+", text)):
        x = 11
    elif "prince" in text:
        x = 12
    elif "lg" in text:
        x = 13
    elif "spc" in text or "hyphone" in text:
        x = 14
    elif "coolpad" in text or "dazen" in text:
        x = 15
    elif "andromax" in text or "prime" in text or "smartfren" in text:
        x = 16
    elif "infinix" in text or "hot s3" in text:
        x = 17
    elif "blaupunkt" in text:
        x = 18
    elif "lava" in text or "iris" in text:
        x = 19
    elif "aldo" in text or bool(re.match(r"a[sf][\-0-9]+", text)):
        x = 20
    elif "huawei" in text or "ascend" in text:
        x = 21
    elif "itab " in text or "advan " in text or "hammer" in text or "ilite" in text:
        x = 22
    elif "le " in text or "letv" in text:
        x = 23
    elif "nexc" in text or "c1000" in text:
        x = 24
    elif "zyrex" in text or bool(re.match(r"z[tcs][\-0-9]+", text)):
        x = 25
    elif "axioo" in text or "venge" in text:
        x = 26
    elif "elephone" in text:
        x = 27
    elif "himax" in text or "htwo" in text or "hi " in text or "hclassic" in text:
        x = 28
    elif "hp" in text or "voicetab" in text or "hewlett" in text:
        x = 29
    elif "nokia" in text:
        x = 30
    elif "nuu" in text:
        x = 31
    elif "cherry" in text:
        x = 32
    elif "mi" in text or "redmi" in text or "xiaomi" in text:
        x = 33
    elif "pixcom" in text:
        x = 34
    elif "mito" in text:
        x = 35
    elif "maxtron" in text or "army" in text or "apollo" in text:
        x = 37
    elif "sony" in text or "xperia" in text:
        x = 38
    elif "im3" in text or "indosat" in text:
        x = 39
    elif "philips" in text or "xenium" in text:
        x = 40
    elif "lenovo" in text or "thinkpad" in text or "ideapad" in text:
        x = 41
    elif "alcatel" in text:
        x= 42
    elif "samsung" in text or "galaxy" in text or "gear" in text:
        x = 43
    elif "doogee" in text:
        x = 45
    elif "vivo" in text or "viv" in text or bool(re.match(r"[yv][0-9]+", text)):
        x = 46
    elif "evercoss" in text or "xtream1" in text or bool(re.match(r"tc[0-9]+", text)):
        x = 47
    elif "strawberry" in text or "trawberry" in text:
        x = 48
    elif "fujitsu" in text:
        x = 50
    elif "blackberry" in text:
        x = 51
    elif "asus" in text or "zenfone" in text:
        x = 52
    elif "oneplus" in text:
        x = 53
    elif "honor" in text:
        x = 54
    elif "oppo" in text or bool(re.match(r"[fa][0-9]+", text)):
        x = 55
    print(x)
    return x


In [8]:
#def run_graph(src, input_layer_name, output_layer_name, num_top_predictions):

def run_graph(src, dest, labels, input_layer_name, output_layer_name, num_top_predictions):
    with tf.Session() as sess:
        # Feed the image_data as input to the graph.
        # predictions  will contain a two-dimensional array, where one
        # dimension represents the input image count, and the other has
        # predictions per class
        #i=0
        
        for j in range(tot):

            file_path = os.path.join(src,test_path[j])
            x = read_text(file_path)
            id_curr = str(test_id[j]) + "_Brand"
            if(x != -1):
                change_val(id_curr,x, df)
                continue
            im=load_image(file_path)
            #img=im.convert('RGB')
            #img.save(os.path.join(dest,test[i]+'.jpg'))
            #image_data=load_image(os.path.join(dest,test[i]+'.jpg'))
            softmax_tensor=sess.graph.get_tensor_by_name(output_layer_name)
            predictions,=sess.run(softmax_tensor, {input_layer_name: im})#image_data})
            if j%50 == 0:
                print(str(j)+ " files validated")
            # Sort to show labels in order of confidence             
            top_k = predictions.argsort()[-num_top_predictions:][::-1]
            for node_id in top_k:
                predicted_label = labels[node_id]
                score = predictions[node_id]
                change_val(id_curr,predicted_label, df)
                #print(predicted_label)
                #print(test[i]+',',predicted_label)
                #outfile.write(f+','+str(predicted_label)+','+str(score)+'\n')
            #i+=1


In [9]:
src='D:\kaggle\\ndsc\\mobile_image.tar\\'
dest=os.path.join('D:\kaggle\\ndsc\mobile_image.tar\\','test_img2')
labels='tmp/output_labels.txt'
graph='tmp/output_graph.pb'
input_layer='DecodeJpeg/contents:0'
output_layer='final_result:0'
num_top_predictions=1
labels = load_labels(labels)
load_graph(graph)
run_graph(src,dest,labels,input_layer,output_layer,num_top_predictions)
df.to_csv('submission_image.csv', encoding='utf-8', index=False)
#run_graph(src,input_layer,output_layer,num_top_predictions)

-1
0 files validated
-1
-1
-1
-1
-1
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
50 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
-1
22
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
100 files validated
-1
-1
-1
-1
-1
-1
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
150 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
37
19
-1
-1
-1
-1
-1
-1
-1
200 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
250 files validated
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
48
-1
-1


33
-1
-1
-1
-1
-1
2450 files validated
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
30
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
13
-1
55
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
38
29
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
2650 files validated
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2700 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
13
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2750 files validated
-

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4900 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
55
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4950 files validated
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
30
-1
-1
-1
-1
-1
2
-1
5000 files validated
-1
-1
-1
23
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5050 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
22
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
5100 files validated
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
33
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
30
-1
-1
-1
-1
-1
-1
-1
-1
5150 files validated
-1
-1
-1
-1
43
-1
29
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1

9
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
7300 files validated
-1
-1
16
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
30
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
7350 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
38
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
43
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
7450 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
38
38
-1
-1
-1
-1
-1
-1
13
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
7500 files validated
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
7550 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


-1
-1
-1
-1
-1
-1
-1
37
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
9750 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
41
-1
-1
-1
33
-1
-1
-1
-1
9800 files validated
-1
-1
2
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
52
-1
-1
-1
-1
2
41
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
9850 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
9900 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
47
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
9950 files validated
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
52
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
10000 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
43
-1
-1
-1
-1

...

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29000 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
29050 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
29
-1
-1
52
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29100 files validated
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
30
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29150 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
13
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
5
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29200 files validated
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29250 files validated
-1
-1
30
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


-1
-1
31400 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
4
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
31450 files validated
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
31500 files validated
-1
-1
-1
-1
-1
9
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
55
-1
-1
31550 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
31650 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
5

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33850 files validated
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
43
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
0
-1
-1
-1
-1
-1
-1
-1
33900 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
43
-1
-1
13
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33950 files validated
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
34000 files validated
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
29
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
55
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
34100 files validated
-1
-1
-1
-1
-1
-1
-1
-1
40
-1
-1
-1

-1
-1
-1
-1
-1
-1
36250 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
4
16
-1
-1
-1
-1
-1
-1
-1
-1
-1
14
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
36300 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
36350 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
36400 files validated
2
-1
-1
-1
-1
-1
52
46
-1
-1
-1
21
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
52
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
36450 files validated
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
36500 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
52
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


-1
-1
-1
42
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
23
-1
38650 files validated
-1
-1
-1
-1
47
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
46
-1
-1
21
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
38700 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
38750 files validated
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
11
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
38800 files validated
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
43
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
38850 files validated
-1
-1
-1
-1
-1
-1
55
-1
2
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
55
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
23
-1
-1
-1
-1
52
-1
38900 files validated
-1
2
-1
-1
-1
-1
-1
-1
-1
33
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


In [12]:
df.to_csv('submission.csv', encoding='utf-8', index=False)

In [11]:
print(df.shape)

(1048581, 2)


In [97]:
change_val('hi',12)

Series([], Name: tagging, dtype: object)


IndexError: index 0 is out of bounds for axis 0 with size 0